In [ ]:
# Import packages (more than needed here)
import os
import glob
import subprocess
import matplotlib.pyplot as plt
import pandas as pd
import math
from numpy import *
from pylab import *
#import pygrib

import pyproj
import metpy.calc as mpcalc
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import wrf
from wrf import (to_np, interplevel, geo_bounds, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords)
import shutil
# Download state and coastline data
states = NaturalEarthFeature(category="cultural", scale="50m",
                          facecolor="none", name="admin_1_states_provinces_shp")
from scipy.interpolate import RegularGridInterpolator
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import matplotlib.ticker as ticker
from scipy.ndimage import gaussian_filter

import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from netCDF4 import Dataset, num2date, date2num
from datetime import datetime

In [ ]:
# set some constants for later use
deltap     = 5000.        # vertical isobaric spacing
resol      = 0.5          # degrees
pref       = 100000.      # Pa
cv         = 717.         # J kg-1 K-1
cp         = 1004.7       # J kg-1 K-1
omega      = 7.292e-5     # Rotation rate s-1       
rd         = 287.04       # J kg-1 K-1
fo         = 1.0e-4       # s-1; assume constant Coriolis parameter (Eliassen 1962)
po         = 100000.      # standard pressure (Pa)
r_earth    = 6.3781e6     # meters (Earth Radius)
grav       = 9.80665      # m/s-2
d2r        = (np.pi)/180. # degrees to radians
rv         = 461.50

def find_nearest(a, val):
    return np.abs(a - val).argmin()

In [ ]:
#First, Specifiy File path
filepath = '/glade/work/cmasiello/wrfout/wrfout_d02*'
datafiles = sorted(glob.glob(filepath))

In [ ]:
#Pressure levels we are considering for our aziumuthal averages
plevs  = [100000.,97500.,95000.,92500.,90000.,87500.,85000.,82500.,80000.,77500.,75000.,72500.,70000.,67500.,65000.,62500.,60000.,
           57500.,55000.,52500.,50000.,47500.,45000.,42500.,40000.,37500.,35000.,32500.,30000.,27500.,25000.,22500.,20000.,17500.,
           15000.,12500.,10000., 7500., 6000.]

nlevs  =len(plevs) #level of interest is 850 hPa, which is the sixth level

print("Number of Pressure levels in the vertical", nlevs)
yplabs = ['']*nlevs

In [ ]:
#Below is azimuthally averaged code for 2D variables!
vt_avg = []    #Wind Speed List
ct_avg = []    #Cloud Top Temperature List
refl_avg = []  #Reflectivity List

for z in range(len(datafiles)):
#for z in range(0,1):
    ncfile = Dataset(datafiles[z])
    wrf_out_data = xr.open_dataset(datafiles[z])   
    Time = wrf.extract_times(ncfile, timeidx=0, method='cat', squeeze=True, cache=None, meta=False, do_xtime=False)
    timestr=(str(Time))
    
    # Set up one time string for plot titles, another for file names
    titletime=(timestr[0:10]+' '+timestr[11:16])
    filetime=(timestr[0:10]+'_'+timestr[11:13])
    print('WRF valid time: ',filetime, titletime, timestr)
    
    # get constant variables
    lats = getvar(ncfile, "XLAT")
    lons = getvar(ncfile, "XLONG") 
    land = getvar(ncfile, "LANDMASK")
    ptop = getvar(ncfile, "P_TOP")

    nx = lats.shape[0]
    ny = lats.shape[1]

    # Get variables 
    z = getvar(ncfile, "z")
    p = getvar(ncfile, "pressure")
    uvmet10 = getvar(ncfile,"uvmet10")
    u10 = uvmet10[0]
    v10 = uvmet10[1]
    cloud_top = getvar(ncfile,"ctt")
    refl = getvar(ncfile,"mdbz")

    # print(refl)
    # print(cloud_top)

    #Next, isolate the variable we care to look at, in this case we will only care about the pressure pertubations
    P_pertubation = wrf_out_data['P']
    surface_P_pertubation = P_pertubation[0,0,:,:]
    #print(surface_P_pertubation)
    
    minpressure = np.min(surface_P_pertubation[:,:])
    minp = minpressure.values

    #Now we know what the lowest pressure is, we have to find the index value for where it occurs
    da = surface_P_pertubation
    p_index = np.argwhere(da.where(da == minp,0).values)
    s_n = p_index[0][0]
    w_e = p_index[0][1]
    
    # Azimuthal average by binning.  Need to specify grid point of storm center, and have arrays for lat, lon
    # Could also set up to specify lat/lon of storm center
    binnum  = 125 #this lets us look at 500 kilometers away from the TC center
    binsize = 1
    dx      = 4000.   #Horizontal grid size of our domain, in meters
    
    ydist, xdist, tcdist, rlonfac, wind_10_abs                         = [np.zeros([nx,ny]) for _ in range(5)]
    ten_m_bin, ct_bin, refl_bin                                        = [np.zeros([nx,ny]) for _ in range(3)]

    thavg = np.zeros(nlevs)
    xlabs = ['']*binnum

    #Now, compute the averages we are interested in
    wind_aavg, ct_aavg, refl_aavg                                      = [np.zeros([int(binnum/binsize)]) for _ in range(3)]

    # define storm center
    ypos = s_n
    xpos = w_e

    # compute distance from each grid point to storm center
    rlonfac[:,:] = 111000.*cos(d2r*lats[:,:])
    ydist[:,:]   = (lats[:,:] - lats[ypos,xpos])*111000
    xdist[:,:]   = (lons[:,:] - (lons[ypos,xpos]))*rlonfac
    tcdist[:,:]  = (xdist[:,:]**2 + ydist[:,:]**2)**0.5
    wind_10_abs[:,:] = sqrt(u10[:,:]**2 + v10[:,:]**2)

    #Below is the code that actually computes the azimuthal information
    for j in range (0, binnum, binsize):
        binmin = j*dx
        binmax = (j+binsize)*dx
        bincnt = int(j/binsize)
        xlabs[j] = str(bincnt*12)

        #now compute the bin infor
        ten_m_bin = np.where(((tcdist > binmin) & (tcdist < binmax)), wind_10_abs, np.nan)
        ct_bin = np.where(((tcdist > binmin) & (tcdist < binmax)), cloud_top, np.nan)
        refl_bin = np.where(((tcdist > binmin) & (tcdist < binmax)), refl, np.nan)

        wind_aavg[bincnt] = np.nanmean(ten_m_bin)
        ct_aavg[bincnt] = np.nanmean(ct_bin)
        refl_aavg[bincnt] = np.nanmean(refl_bin)

    vt_avg.append(wind_aavg)
    ct_avg.append(ct_aavg)
    refl_avg.append(refl_aavg)